# before you go, take a moment to use the best practices for this repository

This code was run with `python 3.9.4`. The packages are listed at the `requirements.txt` file.

I strongly recommend using `pyenv` to maintain multiple versions of python installed locally. 

## install the virtualenv package on your python via pip

at the terminal just run:

`python -m pip install --user virtualenv`

## create your virtual env. 

Once you have your virtual env created, the packages you once install will not affect your local python packages. Which will avoid conflits. A good practice is to have a virtualenv per project.

The community convention is to use the name `venv` for the environment you are about to create. 

to create your virtualven just type the following command at your terminal:

`virtualenv venv` 

and you should see a `/env/` directory inside your project folder or with the given name you choose at the step before.


## step 1: go to your local repository and type on your terminal:

Warning: Now you should active your `virtualenv`. Keep in mind you need to activate everytime you begin to work with the project. Otherwise, the packages you once install on your virtualenv won't be available to your local python.

#### Linux/Macos

Active your virtualenv using the command source, if you are using linux-like bash. If you are using Windows the command can be diffrent. 

`source venv/bin/activate`

If your terminal looks like the above, you are good.

`(venv) dc@blckjzz pegabot-twitter-api % `

#### For Windows Users

`\env\Scripts\activate.bat`


## step 2: once your virtualenv

Now you should have your virtualenv setup. It's time to install the required packages for running this project.

On terminal at the project folder type:

`pip install -r requirements.txt `


## step 3: create your .env file locally

on terminal, inside the project folder just copy and rename the `.env-example` to `.env`

`$ cp .env.example .env`

### Update your virtualenv packages
#### everytime you need to update your requirements.txt file, you should generate an updated version of your `virtualenv` to a requirements.txt file

Go to your terminal and type

`pip freeze > requirements.txt`

After that, you should commit only your `requirements.txt` file instead your virtualenv `/venv` directory (which is ignored by default on this project on .gitignore file)

## Step 4: Using the virtualenv with Jupyter notebooks

For this you should need a package `ipykernel`, which will allow you to link your local venv with your Jupyter notebook. You should do this in order to use the venv and its packages on your notebook. 

At your terminal type:

`source venv/bin/activate` if not activated

`pip install ipykernel` if you have not installed previously. I added this package to the `requirements.txt` file so you should be fine.

`python -m ipykernel install --user --name=${PWD##*/}`

You should see a message like the above:

`Installed kernelspec pegabot-twitter-api in /Users/dc/Library/Jupyter/kernels/pegabot-twitter-api`

Once you hit the command above, you should restart your Jupyter server to load the new Kernel. You may stop the server or just restart the kernel.

Now you have a new kernel for your jupyter notebooks and should indicate which kernel to use in your project.

To do this go to your project, open any file `.ipynb`. At the upper menu bar find `Kernel > Change Kernel > ` and select your new kernel. If you used the same command as above, the kernel should be named as the same as the project, which is `pegabot-twitter-api` in my case.


## Documentação do Tweepy 
`https://docs.tweepy.org/en/stable/`



In [2]:
# load .env variables from local .env file
%load_ext dotenv
%dotenv

In [3]:
# if you wish to reload .env variables
%reload_ext dotenv
%env twitter_api_key

'pFlQlVe3RxDDloIFv3Wq3lv2I'

In [4]:
# project imports listed bellow #

####################################
import os, csv, requests, tweepy
from easydict import EasyDict as edict
import asyncio
####################################

In [5]:
# Loading variables

twitter = {}
twitter['TWITTER_API_KEY'] = %env twitter_api_key 
twitter['TWITTER_API_SECRET'] = %env twitter_api_secret
twitter['TWITTER_API_TOKEN'] = %env twitter_access_token
twitter['TWITTER_API_TOKEN_SECRET'] = %env twitter_access_token_secret

In [9]:
# rewrite the json received from the API to a more readable format
def pegabotTransformResponseJson(pa):
    # recovers all data from twitter data key
    lista = {
        # accessing values from profiles
        'username': pa.profiles[0].username,
        'user_id': pa.twitter_data.user_id,
        'user_name': pa.twitter_data.user_name,
        'created_at': pa.twitter_data.created_at,
        'following': pa.twitter_data.following,
        'followers': pa.twitter_data.followers,
        'number_tweets': pa.twitter_data.number_tweets,
        'mentions': pa.twitter_data.mentions,
        # recoverying values for bot probability
        'result_bot_probability': pa.profiles[0].bot_probability.all,
        'result_bot_probability_info': pa.profiles[0].bot_probability.info,
        # recoverying values for sentiment
        'result_sentiment': pa.profiles[0].language_dependent.sentiment.value,
        'result_network': pa.profiles[0].language_independent.network,
        'result_friends': pa.profiles[0].language_independent.friend,
        'result_temporal': pa.profiles[0].language_independent.temporal,
        'result_user': pa.profiles[0].language_independent.user,
        }
    return edict(lista)

In [23]:
def pegabotAPIRequest(profile, env="local"):
    if env == "prod" or env == "production":
        pegabot_api = "https://backend.pegabot.com.br/botometer"
    else:
        pegabot_api = "http://localhost:9000/botometer"
    try:
        params = {'profile': profile, 'search_for':'profile'}
        response = requests.get(pegabot_api, params)
        # there's a need to treat erros from the Pegabot API.
        # the API should return the profile and a json with an error
        # depending on the error maybe retry the request or just move on to the next line
        if response.status_code == 200:
            jsonResponse = edict(response.json())
            return pegabotTransformResponseJson(jsonResponse)
        else:
            # probably some error
            # usually the user' profile is no longer available
            return edict(response.json())
#        response.raise_for_status()
        # pa.content() # Return the raw bytes of the data payload
        # pa.text() # Return a string representation of the data payload
        # pa.json() # This method is convenient when the API returns JSON
    # Code here will only run if the request is successful
    except requests.exceptions.HTTPError as errh:
        print(errh)
    except requests.exceptions.ConnectionError as errc:
        print(errc)
    except requests.exceptions.Timeout as errt:
        print(errt)
    except requests.exceptions.RequestException as err:
        print(err) 

In [17]:
# loading profiles from csv file
perfiscsv = []
with open('/Users/dc/Downloads/cristian.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    next(csv_reader, None)  # skip the headers
    for row in csv_reader:
        perfiscsv.append(row)

In [18]:
perfiscsv

[['Eustacia251'],
 ['VieiraClever1'],
 ['carolin69045611'],
 ['shenybert'],
 ['VavaDone'],
 ['AlziraRosa10'],
 ['de_novaki'],
 ['jorgebrito798'],
 ['Jjjrr70'],
 ['AlexiGa64214227'],
 ['GaboElSoldado'],
 ['CristianoPeix10'],
 ['Carmena79456352'],
 ['william69907498'],
 ['1azevedo_'],
 ['821Claudia'],
 ['Blanca32780063'],
 ['MarclioDiasVie1'],
 ['Teresap1502m'],
 ['saira24338388'],
 ['anzoateguirojo'],
 ['negro20212021'],
 ['eunicecatuchal'],
 ['Vero716'],
 ['Ezequie01455856'],
 ['HctorRu36854512'],
 ['Joseclemente124'],
 ['VelizMariannys'],
 ['RubnDaroQuione2'],
 ['Marisol91870699'],
 ['judsonline'],
 ['cansinesilvage1'],
 ['Keyler93760290'],
 ['Gonza76507998'],
 ['TiaWhatsapp'],
 ['colonbiyenis'],
 ['lesbia_ferrer'],
 ['38Gabrielgb'],
 ['JuniorP40876328'],
 ['iLazulli'],
 ['Lisbeth31982058'],
 ['ginamarudelgad1'],
 ['FernandoLucca13'],
 ['soyjunior01'],
 ['OBruxoeFM_V'],
 ['DigitalG00'],
 ['RSisartex'],
 ['SamuelH24563067'],
 ['enma_pena'],
 ['jacobbermudez6'],
 ['rainyday75'],
 ['Jesu

In [15]:
res.result_user

0.18920715087381748

In [22]:
analise = {}
count = 0
for perfil in perfiscsv:
    #asyncio.create_task() 
    print(perfil)
    analise = pegabotAPIRequest(perfil)
    print(analise)
    count +=1
    if count >= 50:
        break

['Eustacia251']
{'metadata': {'error': {'request': '/1.1/statuses/user_timeline.json', 'error': 'Not authorized.'}}, 'message': 'Sem permissão para acessar. Usuário pode estar bloqueado/suspendido.'}
['VieiraClever1']
{'metadata': {'error': {'request': '/1.1/statuses/user_timeline.json', 'error': 'Not authorized.'}}, 'message': 'Sem permissão para acessar. Usuário pode estar bloqueado/suspendido.'}
['carolin69045611']
{'metadata': {'error': {'request': '/1.1/statuses/user_timeline.json', 'error': 'Not authorized.'}}, 'message': 'Sem permissão para acessar. Usuário pode estar bloqueado/suspendido.'}
['shenybert']
{'metadata': {'error': {'request': '/1.1/statuses/user_timeline.json', 'error': 'Not authorized.'}}, 'message': 'Sem permissão para acessar. Usuário pode estar bloqueado/suspendido.'}
['VavaDone']
{'metadata': {'error': [{'code': 34, 'message': 'Sorry, that page does not exist.'}]}, 'message': 'Esse usuário não existe'}
['AlziraRosa10']
{'metadata': {'error': {'request': '/1.1/

{'metadata': {'error': {'request': '/1.1/statuses/user_timeline.json', 'error': 'Not authorized.'}}, 'message': 'Sem permissão para acessar. Usuário pode estar bloqueado/suspendido.'}
['colonbiyenis']
{'metadata': {'error': {'request': '/1.1/statuses/user_timeline.json', 'error': 'Not authorized.'}}, 'message': 'Sem permissão para acessar. Usuário pode estar bloqueado/suspendido.'}
['lesbia_ferrer']
{'metadata': {'error': {'request': '/1.1/statuses/user_timeline.json', 'error': 'Not authorized.'}}, 'message': 'Sem permissão para acessar. Usuário pode estar bloqueado/suspendido.'}
['38Gabrielgb']
{'metadata': {'error': {'request': '/1.1/statuses/user_timeline.json', 'error': 'Not authorized.'}}, 'message': 'Sem permissão para acessar. Usuário pode estar bloqueado/suspendido.'}
['JuniorP40876328']
{'metadata': {'error': {'request': '/1.1/statuses/user_timeline.json', 'error': 'Not authorized.'}}, 'message': 'Sem permissão para acessar. Usuário pode estar bloqueado/suspendido.'}
['iLazull

In [24]:
# TODO: transform Pegabot Request async and make multiple request to it.
#profiles = ['blckjzz']
for p in profiles:
    profile = pegabotAPIRequest(p)
    
print(profile)

NameError: name 'profiles' is not defined